<a href="https://colab.research.google.com/github/sergiocostaifes/PPCOMP_DM/blob/main/notebooks/05_feature_engineering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 05_feature_engineering.ipynb — Engenharia de Atributos (5 min)

Objetivo  
Construir atributos derivados para modelagem supervisionada a partir da série temporal contínua (5 minutos) e da segmentação de episódios críticos.

Entradas (artefatos dos Notebooks 03 e 04)

- window_5min_series.parquet
- episodes_detected.parquet

Saídas (artefatos deste Notebook)

- window_5min_features.parquet
- 05_feature_engineering_summary.json

Compatibilidade com o pipeline existente (00 → 04)

Este notebook mantém a granularidade oficial de 5 minutos e:

- Preserva bucket_id como eixo temporal
- Utiliza n_failed como métrica base
- Incorpora is_critical como variável derivada
- Gera atributos rolling e temporais

Features construídas

- Lags (1, 2, 3)
- Rolling mean (1h)
- Rolling std (1h)
- Variação percentual
- Intensidade normalizada

Observações de reprodutibilidade

Bootstrap via Drive + repositório PPCOMP_DM  
Uso de src.paths e ensure_dirs()  
Persistência em 03-features  
Resumo estatístico salvo em 04-reports  
Seed fixa

In [1]:
# ============================================================
# 05_feature_engineering.ipynb
# Engenharia de atributos (5-min)
# Pipeline PPCOMP_DM (Google Cluster Trace)
# ============================================================

# -----------------------------
# 0) BOOTSTRAP (Colab + Repo)
# -----------------------------

from pathlib import Path
import os
import sys
import subprocess
import importlib
import random
import numpy as np
import pandas as pd
import json

SEED = 42
random.seed(SEED)
np.random.seed(SEED)

# Mount Google Drive
if not Path("/content/drive/MyDrive").exists():
    from google.colab import drive
    drive.mount("/content/drive")
else:
    print("[Bootstrap] Google Drive já montado.")

REPO_DIR = Path("/content/drive/MyDrive/Mestrado/PPCOMP_DM")
GITHUB_REPO = "https://github.com/sergiocostaifes/PPCOMP_DM.git"

if not REPO_DIR.exists():
    REPO_DIR.parent.mkdir(parents=True, exist_ok=True)
    print(f"[Bootstrap] Clonando repositório em: {REPO_DIR}")
    subprocess.run(["git", "clone", GITHUB_REPO, str(REPO_DIR)], check=True)
else:
    try:
        print("[Bootstrap] Atualizando repositório (git pull)...")
        subprocess.run(["git", "-C", str(REPO_DIR), "pull"], check=True)
    except Exception as e:
        print("[Bootstrap] Aviso: não foi possível atualizar:", e)

os.chdir(str(REPO_DIR))
print("[Bootstrap] CWD =", os.getcwd())

repo_str = str(REPO_DIR)
if repo_str not in sys.path:
    sys.path.insert(0, repo_str)

importlib.invalidate_caches()

from src.paths import FEATURES_PATH, REPORTS_PATH, ensure_dirs
ensure_dirs()

print("FEATURES_PATH =", FEATURES_PATH)

def log(msg):
    print(f"[05_feature_engineering] {msg}")

# =========================
# 1) Leitura da série base
# =========================

SERIES_FILE = FEATURES_PATH / "window_5min_series.parquet"
EPISODES_FILE = FEATURES_PATH / "episodes_detected.parquet"

assert SERIES_FILE.exists(), f"Arquivo não encontrado: {SERIES_FILE}"
assert EPISODES_FILE.exists(), f"Arquivo não encontrado: {EPISODES_FILE}"

df = pd.read_parquet(SERIES_FILE)
episodes = pd.read_parquet(EPISODES_FILE)

df = df.sort_values("bucket_id").reset_index(drop=True)

assert "bucket_id" in df.columns
assert "n_failed" in df.columns

log(f"Shape série base: {df.shape}")

# =========================
# 2) Criar variável target
# =========================

df["is_critical"] = 0

for _, row in episodes.iterrows():
    df.loc[
        (df["bucket_id"] >= row["start_bucket"]) &
        (df["bucket_id"] <= row["end_bucket"]),
        "is_critical"
    ] = 1

log(f"Total janelas críticas: {df['is_critical'].sum()}")

# =========================
# 3) Construção de features
# =========================

# Lags
df["lag_1"] = df["n_failed"].shift(1)
df["lag_2"] = df["n_failed"].shift(2)
df["lag_3"] = df["n_failed"].shift(3)

# Rolling 1 hora (12 janelas de 5 min)
W = 12
df["rolling_mean_1h"] = df["n_failed"].rolling(W, min_periods=1).mean()
df["rolling_std_1h"] = df["n_failed"].rolling(W, min_periods=1).std().fillna(0)

# Variação percentual
df["pct_change"] = df["n_failed"].pct_change()
df["pct_change"] = df["pct_change"].replace([np.inf, -np.inf], 0).fillna(0)

# Intensidade normalizada
mu = df["n_failed"].mean()
sigma = df["n_failed"].std(ddof=0)
df["zscore_global"] = (df["n_failed"] - mu) / (sigma if sigma > 0 else 1)

# =========================
# 4) Limpeza inicial (remover NaNs de lag)
# =========================

df = df.dropna().reset_index(drop=True)

log(f"Shape após feature engineering: {df.shape}")

# =========================
# 5) Persistência
# =========================

OUT_FILE = FEATURES_PATH / "window_5min_features.parquet"
df.to_parquet(OUT_FILE, compression="snappy", index=False)

summary = {
    "rows": int(len(df)),
    "features": list(df.columns),
    "critical_ratio": float(df["is_critical"].mean()),
    "mu": float(mu),
    "sigma": float(sigma)
}

summary_file = REPORTS_PATH / "05_feature_engineering_summary.json"
summary_file.write_text(json.dumps(summary, indent=2, ensure_ascii=False))

log("Notebook 05 finalizado com sucesso.")
df.head()

Mounted at /content/drive
[Bootstrap] Atualizando repositório (git pull)...
[Bootstrap] CWD = /content/drive/MyDrive/Mestrado/PPCOMP_DM
FEATURES_PATH = /content/drive/MyDrive/Mestrado/02-datasets/03-features
[05_feature_engineering] Shape série base: (8918, 18)
[05_feature_engineering] Total janelas críticas: 200
[05_feature_engineering] Shape após feature engineering: (8914, 26)
[05_feature_engineering] Notebook 05 finalizado com sucesso.


,bucket_id,bucket_start_us,n_events,n_failed,n_machines,n_collections,mean_priority,mean_req_cpus,mean_req_mem,req_cpus_presence_rate,...,event_EVICT_count,event_KILL_count,is_critical,lag_1,lag_2,lag_3,rolling_mean_1h,rolling_std_1h,pct_change,zscore_global
0,15,4500000000,44,10,41,29,249.522727,0.008850,0.003220,1.0,...,0,0,0,11.0,13.0,6.0,10.000000,2.943920,-0.090909,0.416312
1,16,4800000000,22,5,22,19,219.045455,0.011749,0.003593,1.0,...,0,1,0,10.0,11.0,13.0,9.000000,3.391165,-0.500000,-0.135315
2,17,5100000000,28,5,27,22,279.392857,0.010309,0.004613,1.0,...,0,0,0,5.0,10.0,11.0,8.333333,3.444803,0.000000,-0.135315
3,18,5400000000,29,7,29,22,259.517241,0.007249,0.009697,1.0,...,0,2,0,5.0,5.0,10.0,8.142857,3.184785,0.400000,0.085336
4,19,5700000000,36,9,34,27,237.027778,0.010176,0.009284,1.0,...,0,0,0,7.0,5.0,5.0,8.250000,2.964071,0.285714,0.305987


## Achados do Notebook 05 — Engenharia de Atributos

### Entrada e consistência

- Série temporal contínua com 8918 janelas de 5 minutos.
- Após construção de lags e remoção de valores iniciais indefinidos, dataset final com 8914 janelas.
- Total de atributos finais: 26 colunas.

A estrutura temporal foi preservada sem embaralhamento ou vazamento de informação futura.

---

### Distribuição da variável-alvo

- Total de janelas críticas: 200
- Proporção de janelas críticas: ~2,24%

Observação: trata-se de problema naturalmente desbalanceado, coerente com detecção de eventos raros em sistemas distribuídos.

Esse comportamento será considerado nas etapas de modelagem supervisionada.

---

### Features construídas

Foram adicionadas as seguintes classes de atributos:

1. Lags temporais (1, 2 e 3 janelas)
2. Rolling mean (1 hora)
3. Rolling std (1 hora)
4. Variação percentual instantânea
5. Z-score global

Essas features capturam:

- Inércia temporal
- Tendência local
- Volatilidade de curto prazo
- Intensidade relativa ao baseline global

---

### Sanidade estatística

- μ global: 6.2265
- σ global: 9.0641
- Z-score consistente com distribuição assimétrica observada anteriormente.

Rolling std mostra variação dinâmica, permitindo capturar mudanças locais de regime.

---

### Considerações metodológicas

A engenharia de atributos mantém:

- Causalidade temporal
- Reprodutibilidade
- Granularidade fixa de 5 minutos
- Compatibilidade com classificação supervisionada

O dataset gerado (`window_5min_features.parquet`) constitui a base para:

- Rotulagem temporal (BEFORE / DURING / AFTER)
- Treinamento de modelos supervisionados
- Avaliação comparativa de classificadores

---

### Conclusão

O Notebook 05 consolida a transição entre detecção estatística e modelagem preditiva, estabelecendo a matriz de atributos que sustentará os experimentos supervisionados subsequentes.